Import Dependencies

In [2]:
import requests
import time
import json
from unidiff import PatchSet
from phabricator import Phabricator
from datetime import datetime, timedelta
from pprint import pprint
import pandas as pd
import ast

Set API token and repo URL

In [3]:
API_TOKEN = "***REMOVED***"
BASE_URL = "https://phabricator.services.mozilla.com/api/"

Initialize Phabricator client

In [4]:
phab = Phabricator(token=API_TOKEN, host=BASE_URL)
phab.update_interfaces()
phab.user.whoami()

<Result: {'phid': 'PHID-USER-tkqragx363gqgxck66he', 'userName': 'alal', 'realName': 'Ali', 'image': 'https://mozphab-phabhost-cdn.devsvcprod.mozaws.net/file/data/mphx2gyvrtdt6l3r4hqk/PHID-FILE-i4wttqib4xr2hi2b7nz4/8c9c85-alphanumeric_lato-white_A.png-255%2C255%2C255%2C0.7.png', 'uri': 'https://phabricator.services.mozilla.com/p/alal/', 'roles': ['verified', 'approved', 'activated'], 'primaryEmail': 'a.s.salehi.95@gmail.com'}>

Get all the repositories

In [5]:
operation = "diffusion.repository.search"
API_URL = f"{BASE_URL}/{operation}"

payload = {
    "api.token": API_TOKEN,
}

requests.post(API_URL, data=payload).json()

{'result': {'data': [{'id': 37,
    'type': 'REPO',
    'phid': 'PHID-REPO-bjaradtlpdxeajadsmgl',
    'fields': {'name': 'mozilla-beta',
     'vcs': 'hg',
     'callsign': 'BETA',
     'shortName': 'beta',
     'status': 'active',
     'isImporting': False,
     'almanacServicePHID': None,
     'refRules': {'fetchRules': [], 'trackRules': [], 'permanentRefRules': []},
     'defaultBranch': 'default',
     'description': {'raw': 'Source tree for beta releases of Firefox.'},
     'spacePHID': None,
     'dateCreated': 1644349533,
     'dateModified': 1742382423,
     'policy': {'view': 'users', 'edit': 'admin', 'diffusion.push': 'users'}},
    'attachments': {}},
   {'id': 1,
    'type': 'REPO',
    'phid': 'PHID-REPO-saax4qdxlbbhahhp2kg5',
    'fields': {'name': 'mozilla-central',
     'vcs': 'hg',
     'callsign': 'MOZILLACENTRAL',
     'shortName': 'mozilla-central',
     'status': 'active',
     'isImporting': False,
     'almanacServicePHID': None,
     'refRules': {'fetchRules': []

Get revision for a bug

In [5]:
operation = "differential.revision.search"
API_URL = f"{BASE_URL}/{operation}"

payload = {
    "api.token": API_TOKEN,
    "constraints[query]": "1931487"
}

requests.post(API_URL, data=payload).json()

{'result': {'data': [{'id': 229239,
    'type': 'DREV',
    'phid': 'PHID-DREV-742ecttyirpscb3cma4q',
    'fields': {'stackGraph': {'PHID-DREV-742ecttyirpscb3cma4q': []},
     'title': 'Bug 1931487 - Add an MruCache for lang-to-script mappings.',
     'uri': 'https://phabricator.services.mozilla.com/D229239',
     'authorPHID': 'PHID-USER-autti2wxdumepz7idru4',
     'status': {'value': 'published',
      'name': 'Closed',
      'closed': True,
      'color.ansi': 'cyan'},
     'repositoryPHID': 'PHID-REPO-saax4qdxlbbhahhp2kg5',
     'diffPHID': 'PHID-DIFF-xh5y22yks5g64gehbgyl',
     'diffID': '946314',
     'summary': '',
     'testPlan': '',
     'isDraft': False,
     'holdAsDraft': False,
     'dateCreated': 1731773380,
     'dateModified': 1731794654,
     'policy': {'view': 'public', 'edit': 'PHID-PROJ-njo5uuqyyq3oijbkhy55'},
     'bugzilla.bug-id': '1931487',
     'uplift.request': []},
    'attachments': {}}],
  'maps': {},
  'query': {'queryKey': None},
  'cursor': {'limit': 10

Get the diff for a revision

In [23]:
operation = "differential.diff.search"
API_URL = f"{BASE_URL}/{operation}"

payload = {
    "api.token": API_TOKEN,
    "constraints[revisionPHIDs][0]": "PHID-DREV-742ecttyirpscb3cma4q"
}

requests.post(API_URL, data=payload).json()

{'result': {'data': [{'id': 946314,
    'type': 'DIFF',
    'phid': 'PHID-DIFF-xh5y22yks5g64gehbgyl',
    'fields': {'revisionPHID': 'PHID-DREV-742ecttyirpscb3cma4q',
     'authorPHID': 'PHID-USER-autti2wxdumepz7idru4',
     'repositoryPHID': 'PHID-REPO-saax4qdxlbbhahhp2kg5',
     'refs': [{'type': 'base',
       'identifier': '00a3e992ba4fe62cba6bcf381b98128da0bc581a'}],
     'dateCreated': 1731794384,
     'dateModified': 1731794649,
     'policy': {'view': 'public'}},
    'attachments': {}},
   {'id': 946305,
    'type': 'DIFF',
    'phid': 'PHID-DIFF-fmurav5p5edly32whqho',
    'fields': {'revisionPHID': 'PHID-DREV-742ecttyirpscb3cma4q',
     'authorPHID': 'PHID-USER-autti2wxdumepz7idru4',
     'repositoryPHID': 'PHID-REPO-saax4qdxlbbhahhp2kg5',
     'refs': [{'type': 'branch', 'name': 'default'},
      {'type': 'base',
       'identifier': '6c508a387477e3b72db913a9e1761e9a433d06a2'}],
     'dateCreated': 1731773378,
     'dateModified': 1731773380,
     'policy': {'view': 'public'}

Get raw diff and format it

In [ ]:
operation = "differential.getrawdiff"
API_URL = f"{BASE_URL}/{operation}"

payload = {
    "api.token": API_TOKEN,
    "diffID": "946305"
}

diff = requests.post(API_URL, data=payload).json()

response = phab.differential.getrawdiff(diffID='946305')

print(response.response)

patch = PatchSet(diff["result"])

for patched_file in patch:
    print(f"📄 {patched_file.path}")
    for hunk in patched_file:
        print(f"  🔢 Hunk: {hunk.source_start}:{hunk.source_length} → {hunk.target_start}:{hunk.target_length}")
        for line in hunk:
            if line.is_added:
                print(f"    ➕ {line.value.strip()}")
            elif line.is_removed:
                print(f"    ➖ {line.value.strip()}")


diff --git a/gfx/thebes/gfxTextRun.cpp b/gfx/thebes/gfxTextRun.cpp
--- a/gfx/thebes/gfxTextRun.cpp
+++ b/gfx/thebes/gfxTextRun.cpp
@@ -21,10 +21,11 @@
 #include "mozilla/gfx/PathHelpers.h"
 #include "mozilla/intl/Locale.h"
 #include "mozilla/intl/String.h"
 #include "mozilla/intl/UnicodeProperties.h"
 #include "mozilla/Likely.h"
+#include "mozilla/MruCache.h"
 #include "mozilla/ServoStyleSet.h"
 #include "mozilla/Sprintf.h"
 #include "mozilla/StaticPresData.h"
 #include "mozilla/UniquePtr.h"
 #include "mozilla/Unused.h"
@@ -2594,10 +2595,70 @@
   }();
 
   return sScriptTagToCode.get();
 }
 
+static Script ResolveScriptForLang(const nsAtom* aLanguage, Script aDefault) {
+  // Cache for lang-to-script lookups, to avoid constantly needing to parse
+  // and resolve the lang code from scratch.
+  class LangScriptCache
+      : public MruCache<const nsAtom*, std::pair<const nsAtom*, Script>,
+                        LangScriptCache> {
+   public:
+    static HashNumber Hash(const nsAtom* c

Get commmit message for a diff

In [69]:
response = phab.diffusion.commit.search(
    constraints={
        'ids': [946305]
    }
)

pprint(response.response)

{'cursor': {'after': None, 'before': None, 'limit': 100, 'order': None},
 'data': [{'attachments': {},
           'fields': {'auditStatus': {'closed': True,
                                      'color.ansi': None,
                                      'name': 'No Audits',
                                      'value': 'none'},
                      'author': {'email': 'daoshengmu@gmail.com',
                                 'epoch': None,
                                 'identityPHID': 'PHID-RIDT-wxnpicrxe2zpg5vijzs4',
                                 'name': 'Daosheng Mu',
                                 'raw': 'Daosheng Mu <daoshengmu@gmail.com>',
                                 'userPHID': 'PHID-USER-2ljcad6bi6kezqqnumdr'},
                      'committer': {'email': None,
                                    'epoch': 1504119748,
                                    'identityPHID': None,
                                    'name': None,
                                    'raw': 

- Get all the differential revisions (DREV) from last year until now
- For each bug id from bugs_with_added_regressor_info.csv fetch its corresponding differential revision
- For each DREV, get its diff and attach it as a new column to its corresponding bug id from the bugs_with_added_regressor_info.csv

In [14]:
MOZILLA_CENTRAL_PHID = "PHID-REPO-saax4qdxlbbhahhp2kg5"
LIMIT = 100 # page size

# Time range: last 7 days
now = datetime.now()
start_time = now - timedelta(days=365)
start_epoch = int(start_time.timestamp())

all_revisions_list = []
after_cursor = None

while True:

    response = phab.differential.revision.search(
        constraints={
            'createdStart':start_epoch,
            'repositoryPHIDs':[MOZILLA_CENTRAL_PHID]
        },
        order= 'oldest',
        limit= LIMIT,
        after= after_cursor
    )

    data = response['data']
    all_revisions_list.extend(data)

    cursor = response['cursor']
    if cursor['after'] is None:
        break
    after_cursor = cursor['after']


# for rev in all_revisions_list:
#     created_time = datetime.fromtimestamp(rev['fields']['dateCreated']).strftime('%Y-%m-%d %H:%M:%S')
#     print(f"ID: {rev['id']}, PHID: {rev['phid']}, Title: {rev['fields']['title']}, Created: {created_time}")

all_revisions_df = pd.DataFrame(all_revisions_list)
all_revisions_df.to_csv("../datasets/all_differential_revisions.csv", index=False)

In [70]:
all_revisions_df = pd.read_csv("../datasets/all_differential_revisions.csv")
all_revisions_df['fields'] = all_revisions_df['fields'].apply(ast.literal_eval)
all_revisions_list = all_revisions_df.to_dict(orient='records')


bugs_df = pd.read_csv("../datasets/bugs_with_added_regressor_info.csv")
bugs_df['regressed_perf_tests'] = bugs_df['regressed_perf_tests'].apply(ast.literal_eval)
bugs_list = bugs_df.to_dict(orient='records')


# transform to dict for faster lookup
all_revisions_dict = {}

for drev in all_revisions_list:
    drev_bugzilla_bug_id = drev['fields']['bugzilla.bug-id']

    if drev_bugzilla_bug_id:
        all_revisions_dict[int(drev_bugzilla_bug_id)] = drev

bugs_with_drev_list = []

for bug in bugs_list:
    bug_with_drev = bug

    bugzilla_bug_id = bug['bug_id']
    drev_for_bug = all_revisions_dict.get(bugzilla_bug_id)

    bug_with_drev['drev_id'] = None
    bug_with_drev['diff_id'] = None

    if drev_for_bug:
        bug_with_drev['drev_id'] = drev_for_bug['id']
        bug_with_drev['diff_id'] = drev_for_bug['fields']['diffID']

    bugs_with_drev_list.append(bug_with_drev)

bugs_with_drev_df = pd.DataFrame(bugs_with_drev_list)
bugs_with_drev_df.to_csv("../datasets/bugs_with_drev.csv", index=False)


bugs_with_diff_list = []

for bug in bugs_with_drev_list:
    bug_diff_id = bug.get('diff_id')

    if bug_diff_id:

        bug_with_diff = bug

        raw_diff = phab.differential.getrawdiff(diffID = str(bug_diff_id)).response

        bug['raw_diff'] = raw_diff
        bugs_with_diff_list.append(bug_with_diff)


bugs_with_drev_df = pd.DataFrame(bugs_with_diff_list)

bugs_with_drev_df.to_json("../datasets/bugs_with_diff.jsonl", orient="records", lines=True)